In [1]:
from __init__ import * 

In [2]:
device = 'cpu'
mask = torch.ones((1, 1, 1024, 1024))
mri = MRIPixel(mask, device)
sam = SamplerPixel(device)
tv_l2_  = TV_Pixel(device)

In [3]:
c_2 = torch.load('ExpA/data/c_gt.pt')[:, :, :-1, :-1]
c_1 = sam.upsample(c_2)
c_half = sam.upsample(c_1)
c_quad = sam.upsample(c_half)

print(c_2.size(), c_1.size(), c_half.size(), c_quad.size())

torch.Size([1, 1, 512, 512]) torch.Size([1, 1, 1024, 1024]) torch.Size([1, 1, 2048, 2048]) torch.Size([1, 1, 4096, 4096])


In [4]:
c_2_in = c_2
c_1_in = c_1
c_half_in = c_half
c_quad_in = c_quad

print(c_2_in.size(), c_1_in.size(), c_half_in.size(), c_quad_in.size())

torch.Size([1, 1, 512, 512]) torch.Size([1, 1, 1024, 1024]) torch.Size([1, 1, 2048, 2048]) torch.Size([1, 1, 4096, 4096])


In [5]:
# check forward 
mri.set_h(1)
f_hat_1 = mri.H(c_1_in)
mri.set_h(2)
f_hat_2 = mri.H(c_2_in)
mri.set_h(0.5)
f_hat_half = mri.H(c_half_in)
mri.set_h(0.25)
f_hat_quad = mri.H(c_quad_in)

print(torch.mean(torch.abs(f_hat_1-f_hat_2)), torch.mean(torch.abs(f_hat_1-f_hat_half)), torch.mean(torch.abs(f_hat_2-f_hat_half)), torch.mean(torch.abs(f_hat_quad-f_hat_half)))
assert (torch.mean(torch.abs(f_hat_1-f_hat_2))) < 1e-5
assert (torch.mean(torch.abs(f_hat_1-f_hat_half))) < 1e-5
assert (torch.mean(torch.abs(f_hat_2-f_hat_half))) < 1e-5
assert (torch.mean(torch.abs(f_hat_quad-f_hat_half))) < 1e-5

tensor(0., dtype=torch.float64) tensor(1.0361e-10, dtype=torch.float64) tensor(1.0361e-10, dtype=torch.float64) tensor(5.1806e-11, dtype=torch.float64)


In [6]:
# check adjoint

mri.set_h(1)
x = torch.normal(0, 1, (1, 1, 1024, 1024)).double() 
y_prime = torch.normal(0, 1, (1, 1, 1024, 1024)).double() 
y = mri.H(y_prime)

Hx = mri.H(x)
Hty = mri.Ht(y)

Hxy = (torch.conj(Hx) * y).sum()
xHty = (x * Hty).sum()

print(Hxy, xHty)

mri.set_h(2)
x = torch.normal(0, 1, (1, 1, 512, 512)).double() 
y_prime = torch.normal(0, 1, (1, 1, 512, 512)).double() 
y = mri.H(y_prime)

Hx = mri.H(x)
Hty = mri.Ht(y)

Hxy = (torch.conj(Hx) * y).sum()
xHty = (x * torch.conj(Hty)).sum()

print(Hxy, xHty)


mri.set_h(0.5)
x = torch.normal(0, 1, (1, 1, 2048, 2048)).double() 
y_prime = torch.normal(0, 1, (1, 1, 2048, 2048)).double() 
y = mri.H(y_prime)

Hx = mri.H(x)
Hty = mri.Ht(y)

Hxy = (torch.conj(Hx) * y).sum()
xHty = (x * Hty).sum()

print(Hxy, xHty)

mri.set_h(0.25)
x = torch.normal(0, 1, (1, 1, 4096, 4096)).double() 
y_prime = torch.normal(0, 1, (1, 1, 4096, 4096)).double() 
y = mri.H(y_prime)

Hx = mri.H(x)
Hty = mri.Ht(y)

Hxy = (torch.conj(Hx) * y).sum()
xHty = (x * Hty).sum()

print(Hxy, xHty)

tensor(958.1971+3.5527e-14j, dtype=torch.complex128) tensor(958.1971, dtype=torch.float64)
tensor(-708.2475+1.1369e-13j, dtype=torch.complex128) tensor(-708.2475, dtype=torch.float64)
tensor(102.9963+1.1572j, dtype=torch.complex128) tensor(102.9963, dtype=torch.float64)
tensor(60.4443-0.9625j, dtype=torch.complex128) tensor(60.4443, dtype=torch.float64)


In [7]:
# check tv l2
tv_2 = torch.norm(tv_l2_.L(c_2, 2), p=1, dim=1).sum()
tv_1 = torch.norm(tv_l2_.L(c_1, 1), p=1, dim=1).sum()
tv_half = torch.norm(tv_l2_.L(c_half, 0.5), p=1, dim=1).sum()
tv_quad = torch.norm(tv_l2_.L(c_quad, 0.25), p=1, dim=1).sum()

print(tv_1, tv_2, tv_half, tv_quad)

tensor(12341.0388, dtype=torch.float64) tensor(12341.0388, dtype=torch.float64) tensor(12341.0388, dtype=torch.float64) tensor(12341.0388, dtype=torch.float64)
